# Day 17: Trick Shot

You finally decode the Elves' message. HI, the message says. You continue searching for the sleigh keys.

Ahead of you is what appears to be a large ocean trench. Could the keys have fallen into it? You'd better send a probe to investigate.

The probe launcher on your submarine can fire the probe with any integer velocity in the x (forward) and y (upward, or downward if negative) directions. For example, an initial x,y velocity like 0,10 would fire the probe straight up, while an initial velocity like 10,-1 would fire the probe forward at a slight downward angle.

The probe's x,y position starts at 0,0. Then, it will follow some trajectory by moving in steps. On each step, these changes occur in the following order:

The probe's x position increases by its x velocity.
The probe's y position increases by its y velocity.
Due to drag, the probe's x velocity changes by 1 toward the value 0; that is, it decreases by 1 if it is greater than 0, increases by 1 if it is less than 0, or does not change if it is already 0.
Due to gravity, the probe's y velocity decreases by 1.
For the probe to successfully make it into the trench, the probe must be on some trajectory that causes it to be within a target area after any step. The submarine computer has already calculated this target area (your puzzle input).

In [ ]:
record struct Point(int x, int y) {
    public override string ToString()
        => $"{x}/{y}";
    static public Point Zero()
        => new Point(0,0);
    public Point Add(Point to)
        => new Point(x + to.x, y + to.y);
    public Point Drag()
        => new Point(x switch {
                > 0 => x - 1,
                < 0 => x + 1,
                _ => 0,
            }, y - 1);
    public bool In(Target t)
        => x >= t.xMin && x <= t.xMax && y >= t.yMin && y <= t.yMax;
}

record struct Target(int xMin, int xMax, int yMin, int yMax) {
    public IEnumerable<Point> Scan() {
        for (var x = xMin; x <= xMax; x++)
            for (var y = yMin; y<= yMax; y++)
                yield return new Point(x, y);
    }
    public bool Contains(Point p)
        => p.x >= xMin && p.x <= xMax && p.y >= yMin && p.y <= yMax;
}

record struct State(Point pos, Point vel) {
    public State Step()
        => new State(pos.Add(vel), vel.Drag());
}

int MaxHeight(Point initial, Target target) {
    var state = new State(Point.Zero(), initial);
    var yMax = 0;
    
    while (state.pos.x <= target.xMax && state.pos.y >= target.yMin) {
        state = state.Step();
        if (state.pos.y > yMax) yMax = state.pos.y;        
        // Console.WriteLine(state);
        if (target.Contains(state.pos)) return yMax;
    }
    // Console.WriteLine(state);
    return -1;
}

## Input

In [ ]:
using System.IO;
// var input = File.ReadAllText(@"day-17.sample");
var input = File.ReadAllText(@"day-17.input");

var parts = input.Split('=', '.', ',').Where(s => int.TryParse(s, out var r)).Select(s => int.Parse(s)).ToArray();

var target = new Target(parts[0], parts[1], parts[2], parts[3]);

target

xMin,xMax,yMin,yMax
56,76,-162,-134


## Part 1

If you're going to fire a highly scientific probe out of a super cool probe launcher, you might as well do it with style. How high can you make the probe go while still reaching the target area?

Find the initial velocity that causes the probe to reach the highest y position and still eventually be within the target area after any step. What is the highest y position it reaches on this trajectory?

In [ ]:
var range = new Target(
    (int)Math.Floor(Math.Sqrt(target.xMin)),
    (int)Math.Ceiling(Math.Sqrt(2*target.xMax)),
    (int)Math.Floor(Math.Sqrt(-target.yMax)),
    -target.yMin);
    
display(range);

var heights = range.Scan().Select(p
        => (p, MaxHeight(p, target))
    ).ToArray();
    
heights.Max(t => t.Item2)


xMin,xMax,yMin,yMax
7,13,11,162


13041

## Part 2

Maybe a fancy trick shot isn't the best idea; after all, you only have one probe, so you had better not miss.

To get the best idea of what your options are for launching the probe, you need to find every initial velocity that causes the probe to eventually be within the target area after any step.

How many distinct initial velocity values cause the probe to be within the target area after any step?

In [ ]:
var range = new Target(
    (int)Math.Floor(Math.Sqrt(target.xMin)),
    target.xMax,
    target.yMin,
    -target.yMin);
    
display(range);

var heights = range.Scan().Select(p
        => (p, MaxHeight(p, target))
    ).ToArray();

heights.Count(i => i.Item2 >= 0)

xMin,xMax,yMin,yMax
7,76,-162,162


1031